# Nonlinear Regression

How do we estimate the parameters for a nonlinear regression problem? For example, suppose we have data that we believe are represented by a power law with two parameters, e.g.

\begin{equation}
\hat{y}(t) = \hat{\beta}_1 t^{\hat{\beta}_2}.
\end{equation}

This model is not linear in the parameters since the derivative of $\hat{y}$ with respect to $\hat{\beta}_1$ depends on $\hat{\beta}_2$. Morover, the derivative with respect of $\hat{\beta}_2$ depends on both $\hat{\beta}_1$ and $\hat{\beta}_2$. 

\begin{align}
\frac{\partial \hat{y}}{\partial \hat{\beta}_1} = & t^{\hat{\beta}_2} \\
\frac{\partial \hat{y}}{\partial \hat{\beta}_2} = & \hat{\beta}_1 t^{\hat{\beta}_2} \log(t) \\
\end{align}

Power law models like this are often used to describe power spectra of ocean variables such as velocity and temperature. For example, the energy of some processes are postulated to have a frequency dependence of $\omega^{-2}$.

In [ ]:
import colorednoise as cn
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as sig

We can generate synthetic data with a -2 spectral slope using the `colorednoise` package.

In [ ]:
n_data = 2**10
t, dt = np.linspace(0, 100, n_data, retstep=True)
u = cn.powerlaw_psd_gaussian(2, n_data, random_state=1351)

fig, ax = plt.subplots()
ax.plot(t, u)
ax.set_xlabel("t")
ax.set_ylabel("u(t)")

The power spectrum may have a -2 dependence. It is difficult to see with a linear axis.

In [ ]:
om, Pu = sig.welch(u, fs=1 / dt, window="hann", nperseg=256, noverlap=128)

fig, ax = plt.subplots()
ax.plot(om, Pu)
ax.set_xlabel("$\omega$")
ax.set_ylabel("$P_u(\omega)$")

Transforming to a log-log axis makes the power law much more obvious. 

In [ ]:
fig, ax = plt.subplots()
ax.loglog(om, Pu)
ax.set_xlabel(r"$\omega$")
ax.set_ylabel(r"$P_u(\omega)$")

f_guide = np.array([1e0, 2e0])
ax.loglog(f_guide, f_guide ** (-2), "k--")
ax.annotate(fr"$\omega^{-2}$", xy=(f_guide[0], f_guide[0] ** (-2)))

Plotting on logarithmic axes is equivalet to transforming the model by taking the logarithm. 

\begin{equation}
\log(\hat{y}(t)) = \log(\hat{\beta}_1) + \hat{\beta}_2 \log(t)
\end{equation}

In this case, taking the logarithm transforms the problem into a linear regression problem. We could rewrite the equation above as

\begin{equation}
z = b + m x 
\end{equation}

where $z = \log(\hat{y}(t))$, $b = \log(\hat{\beta}_1)$, $m=\hat{\beta}_2$ and $x = \log(t)$.  It is now the univariate 2-parameter linear model. The parameters can be estimated using the linear regression techniques seen before.


In [ ]:
X = np.column_stack((np.ones_like(om[1:]), np.log(om[1:])))
log_params = np.linalg.lstsq(X, np.log(Pu[1:]), rcond=None)[0]

beta_1 = np.exp(log_params[0])
beta_2 = log_params[1]
y_log_fit = beta_1 * om[1:]**beta_2

fig, ax = plt.subplots()
ax.loglog(om[1:], Pu[1:])
ax.loglog(om[1:], y_log_fit, "k", label=f"${beta_1:.3f} \omega^{{{beta_2:.2f}}}$")
ax.set_xlabel("$\omega$")
ax.set_ylabel("$P_u(\omega)$")
ax.legend()

One key assumption in this transformation is that the errors are multiplicative, not additive. The linear regression model we saw before had additive errors, e.g. $y = \hat{y} + \epsilon$, thus allowing us to minimize $\langle \epsilon^2 \rangle = \langle (y - \hat{y})^2 \rangle$. Taking the logarithm of the addative error model impedes our ability minimize the error. How can we rearrange $\log(\hat{y} + \epsilon)$ to minimize $\epsilon$? If the errors are multiplicative, meaning $y = \epsilon \hat{y}$, then taking the logarithm produces an additive error term that can be minimized.  

Another issue when applying linear regression to the logarithm of data is the asymmetric weighting of data in the fit. Small values are given more weight. In other words, you may not fit large values well. A model difference of 1 in $\log_{10}$ space is equivalent to a multiplicative difference of $10 \times$ in linear space. Thus, a misfit of 1 in $\log_{10}$ at a y data point that has a value of 100 could mean a difference of $10 \times 100 - 100 = 900$ or $100/10 - 100 = -90$. The same $\log_{10}$ difference of 1 on data with value 1 could mean a difference of +9 or -0.9. Much smaller! Be wary. 

What about a nonlinear model that cannot be reduced to a linear problem? E.g. $\hat{y}(t) = \hat{\beta}_1 \cos(\hat{\beta}_2 t)$. Or what if we don't want to reduce our power law to a linear regression problem because we are not sure that the assumptions hold? Then we need to use nonlinear regression. 

Minimizing the mean square error $\langle \epsilon^2 \rangle$ is still the main goal of nonlinear regression. However, now we must take a new approach. We are not able to derive an analytical solution, so we rely on a numerical solution. 

## Nonlinear Least Squares

Several methods are available to minimize a nonlinear least squares problem. We will consider the Gauss-Newton algorithm. The method relies on a first guess of the parameters, followed by successive iterations to hone in on the true values. The goal is to estimate all the $M$ parameters,

\begin{equation}
\boldsymbol{\hat{\beta}} =
\begin{bmatrix}
\hat{\beta}_1 \\
\hat{\beta}_2 \\
\vdots \\
\hat{\beta}_M
\end{bmatrix},
\end{equation}

by minimizing the square error between the data and the model. At each iteration $k$ we update our initial parameter guess by some amount $\Delta \boldsymbol{\hat{\beta}}$ to use in the next iteration $k+1$,

\begin{equation}
\boldsymbol{\hat{\beta}}^{k+1} = \boldsymbol{\hat{\beta}}^k + \Delta \boldsymbol{\hat{\beta}}^k
\end{equation}

The iteration ends when we are satisfied that we are close to the best parameter estimate. How do we find $\Delta \boldsymbol{\hat{\beta}}^k$?

First, we recognize that the model is a function of the independent variable at times $n = 1, 2, ... N$, and the parameters, e.g. $\hat{y} = \hat{y}(t_n, \boldsymbol{\hat{\beta}})$. Because the model is nonlinear it cannot be written as a matrix of inputs multiplying a matrix of parameters. However, we can linearize the model about the parameters using a Taylor expansion,.

\begin{equation}
\hat{y}(t_n, \boldsymbol{\hat{\beta}}^k + \Delta \boldsymbol{\hat{\beta}}^k) \approx \hat{y}(t_n, \boldsymbol{\hat{\beta}}^k) + \boldsymbol{J} \Delta\boldsymbol{\hat{\beta}}^k,
\end{equation}

where $\boldsymbol{J}(t_n, \boldsymbol{\hat{\beta}}^k)$ is the $N\times M$ Jacobian matrix,

\begin{equation}
\boldsymbol{J} =
\begin{bmatrix}
\dfrac{\partial \hat{y}(t_1, \boldsymbol{\hat{\beta}}^k)}{\partial \hat{\beta}_1} & \dfrac{\partial \hat{y}(t_1, \boldsymbol{\hat{\beta}}^k)}{\partial \hat{\beta}_2} & \cdots & \dfrac{\partial \hat{y}(t_1, \boldsymbol{\hat{\beta}}^k)}{\partial \hat{\beta}_M} \\[1.5em]
\dfrac{\partial \hat{y}(t_2, \boldsymbol{\hat{\beta}}^k)}{\partial \hat{\beta}_1} & \dfrac{\partial \hat{y}(t_2, \boldsymbol{\hat{\beta}}^k)}{\partial \hat{\beta}_2} & \cdots & \dfrac{\partial \hat{y}(t_2, \boldsymbol{\hat{\beta}}^k)}{\partial \hat{\beta}_M} \\[1.5em]
\vdots & \vdots & \ddots & \vdots \\[1.5em]
\dfrac{\partial \hat{y}(t_N, \boldsymbol{\hat{\beta}}^k)}{\partial \hat{\beta}_1} & \dfrac{\partial \hat{y}(t_N, \boldsymbol{\hat{\beta}}^k)}{\partial \hat{\beta}_2} & \cdots & \dfrac{\partial \hat{y}(t_N, \boldsymbol{\hat{\beta}}^k)}{\partial \hat{\beta}_M}
\end{bmatrix}.
\end{equation}

We are trying to minimize

\begin{equation}
\langle (y - \hat{y})^2 \rangle.
\end{equation}